In [ ]:
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.corpus import inaugural

### Read the inaugural corpus into a dataframe with 2 columns: docid and doc
**docid = simple index**

**doc = raw text**

In [ ]:
df = pd.DataFrame()

i=0
for fileid in inaugural.fileids():
    df = df.append({'docid':i, 'doc':inaugural.raw(fileid)}, ignore_index=True)
    i += 1

In [ ]:
df.head()

In [ ]:
stop_words = set(stopwords.words('english'))
new_stops = ['"', '(', ')', ',', '-', '--', '.', '."', '14th', ':', ';', "'s",\
             '!', '";', '"?', '?"', '$', "'", '),', ',"', '.)', '', '...', '....',\
             '[', '?', ']', '000']
stop_words.update(new_stops)

### scikit-learn contains a very convenient TF-IDF function
see https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# initialize TFidfVectorizer, including stop words
tfidf_vectorizer=TfidfVectorizer(use_idf=True,  stop_words=stop_words )
 
# fit_transform returns the tf-idf transformed corpus in a document-by-term matrix
X=tfidf_vectorizer.fit_transform(df['doc'])

In [ ]:
# shape?


In [ ]:
# call stop_words to inspect stop words
tfidf_vectorizer.stop_words

In [ ]:
# call get_feature_names() to see retained terms
tfidf_vectorizer.get_feature_names()

### Compute the truncated SVD
see https://scikit-learn.org/stable/modules/decomposition.html#truncated-singular-value-decomposition-and-latent-semantic-analysis

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)

In [ ]:
svd.fit(X)

**With the SVD computed, we can now extract the factors**

`svd.transform(X)` produces $U * \Sigma$

`svd.components_` generates $V^T$

`svd.singular_values_` generates $\Sigma$ in vector form

**With some math we can get $U$, along with $\Sigma$ and $V^T$**

In [ ]:
U  = svd.fit_transform(X) / svd.singular_values_
S  = svd.singular_values_
VT = svd.components_

### Looking at $\sigma_i, i=1,\ldots,k$, we can see the diminishing signal as $k$ increases

In [ ]:
import matplotlib.pyplot as plt

plt.plot([x for x in range(1,len(S)+1)], S, 'ro')
plt.show()

### To understand the meaning of each _feature vector_ we can look at the terms with largest positive values
`argsort()` when applied to a list sorts by value

In [ ]:
(-VT[0]).argsort()[:5]

**for each singular vector (feature vector), find the index of the 5 largest terms, and extract the corresponding terms**

In [ ]:
terms = tfidf_vectorizer.get_feature_names()
for k in range(0,5):
    for i in (-VT[k]).argsort()[:5]:
        print(k,terms[i])

---
### Let's look at performing IR with a query

In [ ]:
query = 'peace and prosperity and love and care'

**We need to transform the query using the corpus tfidf**

`sklearn.feature_extraction.text.TfidfVectorizer` has a function `transform()` that does exaclty this

In [ ]:
query_xform = tfidf_vectorizer.transform([query])

In [ ]:
print(query_xform)

**Now we project the transformed query into the document-by-feature space**

Let's invert $\Sigma$ by first converting the vector to a diagonal matrix with `np.diag()`
and then inverting it using `np.linalg.inv()`

In [ ]:
Sinv = np.linalg.inv(np.diag(S))

In [ ]:
Sinv

In [ ]:
1/3.481631

In [ ]:
np.matmul(np.diag(S),Sinv)

In [ ]:
Z = np.matmul(np.transpose(VT),Sinv)

In [ ]:
Z

In [ ]:
qk = np.matmul(query_xform.toarray(),Z)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity(qk,qk)

In [ ]:
U[0]

In [ ]:
maxindex = -1
maxcos   = 0
it = np.nditer(U, flags=['f_index'])

for i in range(U.shape[0]):
    d=U[i]
    cos = cosine_similarity([d],qk)
    print(i,cos,maxcos,maxindex)
    if cos>maxcos:
        maxcos=cos
        maxindex=i

In [ ]:
print(maxcos, maxindex)

In [ ]:
pd.set_option('display.max_colwidth', -1)
print(df[df['docid']==39])

In [ ]:
df[]